In [2]:
import os
from os.path import join as pjoin
from scipy import misc
from PIL import Image
import dlib
import cv2
from imutils import face_utils
import time
import shutil

In [3]:
root_folder = 'C:\\Users\\linpeisen\\AppData\\Local\\Packages\\CanonicalGroupLimited.UbuntuonWindows_79rhkp1fndgsc\\LocalState\\rootfs\\home\\linpeisen'
os.chdir(root_folder)

In [6]:
def hisEqulColor(img):  
    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)  
    channels = cv2.split(ycrcb)  
    cv2.equalizeHist(channels[0], channels[0]) #equalizeHist(in,out)  
    cv2.merge(channels, ycrcb)  
    img_eq=cv2.cvtColor(ycrcb, cv2.COLOR_YCR_CB2BGR)  
    return img_eq

In [7]:
def load_data(data_dir,img_dir):  # data_dir是lfw数据集路径
    count = 0
    face_detect = dlib.get_frontal_face_detector()
    new_dimension = (128,128)
    # (32,32)
    for guy in os.listdir(data_dir):  
        person_dir = pjoin(data_dir, guy)  # lfw中文件夹的路径
        for i in os.listdir(person_dir):
            image_dir = pjoin(person_dir, i)  # lfw中每个文件夹中图片的路径
            frame = cv2.imread(image_dir)  # 读取lfw中每个文件夹中的图片 
            img = hisEqulColor(frame)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            rects = face_detect(gray, 1)
            cropped_face = None
            for (i, rect) in enumerate(rects):
                (x, y, w, h) = face_utils.rect_to_bb(rect)
                cropped_face =  frame[y:y+h, x:x+w]
            try:
                if cropped_face is not None:
                    resized_face = cv2.resize(cropped_face, new_dimension)
                    path = img_dir + str(count) + '.jpg'
                    cv2.imwrite(path, resized_face)
                    count = count+1
            except:
                continue
            
        if count >= 500:
            break
  
    return 

In [8]:
img_dir = 'trainingData0\\'  # 自己单独建的文件夹, 用于存放从lfw读取的图片
data_dir = 'lfw_funneled'
if (os.path.exists(img_dir) == False):
    os.mkdir(img_dir)
load_data(data_dir,img_dir)

In [8]:
def HOGdata(video = 0,save_path= ''):
    capture = cv2.VideoCapture(video)
    num_faces = 0
    new_dimension = (128, 128)
    # (32,32)
    face_detect = dlib.get_frontal_face_detector()
    response, frame = capture.read()
    while response:
        img = hisEqulColor(frame)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        rects = face_detect(gray, 1)
        try:
            for (i, rect) in enumerate(rects):
                (x, y, w, h) = face_utils.rect_to_bb(rect)
                cropped_face =  frame[y:y+h, x:x+w]
                num_faces += 1
                # resizing found face to new dimensions
                resized_face = cv2.resize(cropped_face, new_dimension)

            # Saving the training data
                path = save_path + str(num_faces) + '.jpg'
    #             cv2.imwrite(path, resized_face) original
                cv2.imwrite(path, resized_face)
        except:
            response, frame = capture.read()
            continue

        if video == 0:
            cv2.imshow('Video', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        response, frame = capture.read()

    capture.release()
    cv2.destroyAllWindows()

    print ('Training data done')

In [10]:
over_write = True
folder = 'trainingData1'
# if over_write:
#     if (os.path.exists(folder) == True):
#         shutil.rmtree(folder)   
#         time.sleep(5)
        
if (os.path.exists(folder) == False):
    os.mkdir(folder)
HOGdata(video = 'test_video//original.mp4',save_path='{}/'.format(folder)) 

Training data done
